In [48]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim

In [49]:
df_raw = pd.read_csv('preprocessed.csv')

In [50]:
size = 0.2

# obtain training indices that will be used for validation
# split by test size
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=size, random_state=42)
for train_idx, test_idx in split.split(df_raw, df_raw['shop_id']):
    strat_train_set = df_raw.loc[train_idx]
    strat_test_set = df_raw.loc[test_idx]

# make train/test set
dfx = strat_train_set[['botem_avg',
       'botem_std', 'weekend', 'visitors', 'DP', 'SERVICE',
       'DP_ratio', 'SERVICE_ratio']].values
df_test_x = strat_test_set[['botem_avg',
       'botem_std', 'weekend', 'visitors', 'DP', 'SERVICE',
       'DP_ratio', 'SERVICE_ratio']].values

# target label
dfy = strat_train_set[['wifi_botem_weight']].values
df_test_y = strat_test_set[['wifi_botem_weight']].values

In [54]:
[i, j
for i, j
in zip(dfx, dfy)]

SyntaxError: invalid syntax (<ipython-input-54-568ebcf7150a>, line 2)

In [47]:
torch.tensor([dfx, dfy])

ValueError: expected sequence of length 8 at dim 2 (got 1)

In [41]:
train_loader = torch.utils.data.DataLoader(data_test, batch_size=64, shuffle=True)
# train_value_loader = torch.utils.data.DataLoader(dfy, batch_size=64, shuffle=True)

TypeError: object of type 'zip' has no len()

In [30]:
dataiter = iter(train_loader)
valueiter = iter(train_loader)
data = dataiter.next()
value = valueiter.next()

In [31]:
data[0]

tensor([  2.5952,   0.2421,   0.0000, 199.0000, 121.0000, 101.0000,   0.6080,
          0.5075], dtype=torch.float64)

In [32]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)

        self.bn = nn.BatchNorm1d(128)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleMLP()
print(model)

SimpleMLP(
  (fc1): Linear(in_features=8, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleMLP(
  (fc1): Linear(in_features=8, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [34]:
creterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 20
steps = 0

model.train()

for epoch in range(epochs):
    train_loss = 0

    for data in trainloader:
        data = data.to(device)
        optimizer.zero_grad()

        log_prob = model(image)
        log_loss = criterion(log_prob, value)
        log_loss.backward()
        optimizer.step()

        train_loss += log_loss.item()
    train_loss = train_loss/len(trainloader)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))